#### Import important libraries

In [5]:
# Import important libraries

import pandas as pd

#Read reports
orderReport = pd.read_csv('Order Report.csv')
skuMaster = pd.read_csv('SKU Master.csv')
pinCode = pd.read_csv('pincodes.csv')
invoice = pd.read_csv('Invoice.csv')
courierRates = pd.read_csv('Courier Company - Rates.csv')

#### View  Dataset Reports

In [6]:
# View part of the reports

print(f"\n\nOrder Report\n\n {orderReport.head()} \n")
print(f"\nSKU Master\n\n {skuMaster.head()} \n")
print(f"\nPin Codes Mapping\n\n {pinCode.head()} \n")
print(f"\nInvoices Report\n\n {invoice.head()} \n")
print(f"\nCourier Rate Report\n\n {courierRates.head()} \n")



Order Report

    ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN 


SKU Master

              SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN 


Pin Codes Mapping

    Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             12

#### Look for Blanks or Missing Values

In [7]:
# Look at the missing values
print(f"\nMissing Values in the Website Order Report:\n\n{orderReport.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website SKU Master Report:\n\n{skuMaster.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website Invoices Report:\n\n{invoice.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website Pincode Mapping Report:\n\n{pinCode.isnull().sum()} \n")
# Look at the missing values
print(f"\nMissing Values in the Website Courier Rates Report:\n\n{courierRates.isnull().sum()} \n")


Missing Values in the Website Order Report:

ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64 


Missing Values in the Website SKU Master Report:

SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64 


Missing Values in the Website Invoices Report:

AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64 


Missing Values in the Website Pincode Mapping Report:

Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64 


Missing Values in the Website Courier Rates Report:

fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_a

#### Data Cleaning

In [8]:
# Get rid of unnamed columns from the SKU Master DataFrame
orderReport = orderReport.drop(columns = ['Unnamed: 3', 'Unnamed: 4'])

# Get rid of unnamed columns from the SKU Master DataFrame
skuMaster = skuMaster.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

# Get rid of unnamed columns from the Pincode Mappping Datafram 
pinCode = pinCode.drop(columns = ['Unnamed: 3', 'Unnamed: 4'])                               

##### Data Merging

In [9]:
# Merge the Order Report and the SKU Report
merge_order_sku = pd.merge(orderReport, skuMaster, on='SKU')
# Display the top five rows in the merged report
print(merge_order_sku.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001827036  8904223819093        1.0         150
2     2001827036  8904223819109        1.0         100
3     2001827036  8904223818430        1.0         165
4     2001827036  8904223819277        1.0         350


In [10]:
# Rename the 'ExternalOrderNo' to 'Order Id' as in other datasets

merge_order_sku = merge_order_sku.rename(columns = {'ExternOrderNo': 'Order ID'})
print(merge_order_sku.head())

     Order ID            SKU  Order Qty  Weight (g)
0  2001827036  8904223818706        1.0         127
1  2001827036  8904223819093        1.0         150
2  2001827036  8904223819109        1.0         100
3  2001827036  8904223818430        1.0         165
4  2001827036  8904223819277        1.0         350


In [11]:
# Map the courier invoice and pincode mapping based on Pincode
# First drop duplicates from the pincode

abc_courier = pinCode.drop_duplicates(subset=['Customer Pincode'])
courier_abc = invoice[['Order ID', 'Customer Pincode', 'Type of Shipment']]
pincodes = courier_abc.merge(abc_courier,on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


In [12]:
# Now we merge pincodes with the main dataframe
merged_pincodes = merge_order_sku.merge(pincodes,on='Order ID')
#print(merged_pincodes)

In [13]:
# Calculate the weight into kilograms from grams
# 1kg = 1000g; The number of grams is divided by 1000
merged_pincodes['Weights (Kgs)'] = merged_pincodes['Weight (g)'] / 1000

# for i in merged_pincodes['Weight (g)']:
#     print(merged_pincodes['Weights (Kgs)'].head())

##### Calculating Weight Clusters

In [14]:
def weight_cluster(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

merged_pincodes['Weight Cluster (KG)'] = merged_pincodes['Weights (Kgs)'].apply(weight_cluster)
invoice['Weight Slab Charged by Courier Company']=(invoice['Charged Weight']).apply(weight_cluster)

In [15]:
# Renaming columns to create the desired datafram

invoice = invoice.rename(columns={'Zone':'Delivery Zone Charged by the Courier Company'})
merged_pincodes = merged_pincodes.rename(columns={'Zone':'Delivery Zone as per ABC'})
merged_pincodes = merged_pincodes.rename(columns={'Weight Cluster (KG)':'Weight Cluster as per ABC'})
print(merged_pincodes.head(2))

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone as per ABC  Weights (Kgs)  \
0  Forward charges             121003                        e          0.127   
1  Forward charges             121003                        e          0.150   

   Weight Cluster as per ABC  
0                        0.5  
1                        0.5  


#### Calculating the Total Costs for Sending and Delivering a Package

In [16]:
# Expected Costs

total_cost = []

for _, row in merged_pincodes.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone as per ABC']
    fwd_fixed = courierRates.at[0, fwd_category + '_fixed']
    fwd_additional= courierRates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone as per ABC']
    rto_fixed = courierRates.at[0, rto_category + '_fixed']
    rto_additional = courierRates.at[0, rto_category + '_additional']

    weight_cluster = row['Weight Cluster as per ABC']
    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_cluster - 0.5) / 0.5)
        total_cost.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_cluster - 0.5) / 0.5)
        total_cost.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_cost.append(0)

merged_pincodes["Expected Charge as per ABC"] = total_cost # Save the cost to pincodes
print(merged_pincodes.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0  2001827036  8904223818706        1.0         127            173213   
1  2001827036  8904223819093        1.0         150            173213   
2  2001827036  8904223819109        1.0         100            173213   
3  2001827036  8904223818430        1.0         165            173213   
4  2001827036  8904223819277        1.0         350            173213   

  Type of Shipment  Warehouse Pincode Delivery Zone as per ABC  Weights (Kgs)  \
0  Forward charges             121003                        e          0.127   
1  Forward charges             121003                        e          0.150   
2  Forward charges             121003                        e          0.100   
3  Forward charges             121003                        e          0.165   
4  Forward charges             121003                        e          0.350   

   Weight Cluster as per ABC  Expected Charge as per ABC  
0              

In [17]:
# we merge it with the courier invoice to display the final dataframe

merged_output = merged_pincodes.merge(invoice, on="Order ID")
print(merged_output.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8904223818706        1.0         127              173213   
1  2001827036  8904223819093        1.0         150              173213   
2  2001827036  8904223819109        1.0         100              173213   
3  2001827036  8904223818430        1.0         165              173213   
4  2001827036  8904223819277        1.0         350              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone as per ABC  \
0    Forward charges               121003                        e   
1    Forward charges               121003                        e   
2    Forward charges               121003                        e   
3    Forward charges               121003                        e   
4    Forward charges               121003                        e   

   Weights (Kgs)  Weight Cluster as per ABC  Expected Charge as per ABC  \
0          0.127                        0.5          

In [18]:
###Calculate the difference between the charges and expected charges for each order
df_diff = merged_output
df_diff['Difference (RS.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']
df_new_charge = df_diff[['Order ID', 'Difference (RS.)', 'Expected Charge as per ABC']]
print(df_new_charge.head())

     Order ID  Difference (RS.)  Expected Charge as per ABC
0  2001827036              61.3                        56.6
1  2001827036              61.3                        56.6
2  2001827036              61.3                        56.6
3  2001827036              61.3                        56.6
4  2001827036              61.3                        56.6


In [19]:
# Summary of the accuracy of the B2B charges based on the charged prices and  expected prices

# Calculate the total orders in each category
total_correctly_charged = len(df_new_charge[df_new_charge['Difference (RS.)'] == 0])
total_overcharged = len(df_new_charge[df_new_charge['Difference (RS.)'] > 0])
total_undercharged = len(df_new_charge[df_new_charge['Difference (RS.)'] < 0])

# Calculate the total amount in each category
amt_overcharged = abs(df_new_charge[df_new_charge['Difference (RS.)'] > 0]['Difference (RS.)'].sum())
amt_undercharged = df_new_charge[df_new_charge['Difference (RS.)'] < 0]['Difference (RS.)'].sum()
amt_correctly_charged = df_new_charge[df_new_charge['Difference (RS.)'] == 0]['Expected Charge as per ABC'].sum()

# Summary DataFrame
summary = {'Description': ['Total Orders Where ABC has been correctly charged',
                           'Total Orders where ABC has been overcharged',
                           'Total orders where ABC has been undercharged'],
           'Count': [total_correctly_charged, total_overcharged, total_undercharged],
           'Amount (Rs.)': [amt_correctly_charged, amt_overcharged, amt_undercharged]}
df_summary = pd.DataFrame(summary)
print(df_summary)

                                         Description  Count  Amount (Rs.)
0  Total Orders Where ABC has been correctly charged     12         507.6
1        Total Orders where ABC has been overcharged    382       33750.5
2       Total orders where ABC has been undercharged      7        -165.2


In [22]:
# Visualize the proportion of errors
import plotly.graph_objects as go
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='label+percent',
                            hole=0.4))
fig.update_layout(title='Proportion')
fig.show()